# Perceiver Autoencoder

In this notebook, I prototype the **full** version of the Perceiver autoencoder. This includes the training loop and any data management that needs to occur with this. Disk-memory-GPU management of data will not be performed, however. 

**Goals:**
Fully adjustable + parameterizable Perceiver for Predictive Coding.
 - **Encoder**: Adjustable encoder shape, # re-exposures as data is compressed. 
	 - Let's avoid having a bunch of intermediate {#token, token dim} sizes between the original byte array and the final latent state. 
	 - The only adjustment should be the # re-exposures -- i.e., the number of different $\mathbb{R}^{(M \times D)} \to \mathbb{R}^{(N\times C)}$ encoders there are that re-query the byte array using the current latent estimate. 
 - **Latent-latent**: Number of distinct blocks, number of block repeats between new information exposure. 
 - **Decoder**: 